# Data Generation (Supervised Learning)

## Import Packages

In [2]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [3]:
case = pp.networks.case30()
number_of_bus = case.bus.shape[0]

Freq = 50           ## Frequency Hz
S_base = 100
load_bus_number = case.load.loc[:,'bus']

### Z_base

V_base = np.zeros((case.line.shape[0],1))
for  i in range(case.line.shape[0]):
    V_base[i,0] = case.bus.loc[case.bus.index[case.bus.loc[:,'name']==case.line.loc[i,'to_bus']],'vn_kv']

Z_base = np.square(V_base)/S_base

### Data_form_1: PG PL QL  
P_Gen = np.zeros((case.bus.shape[0],1))
Load = np.zeros((case.bus.shape[0],2))


P_Gen[case.gen.loc[:,'bus']] = np.array([case.gen.loc[:,'p_mw']]).transpose()
Load[case.load.loc[:,'bus'],0] = np.array([case.load.loc[:,'p_mw']])
Load[case.load.loc[:,'bus'],1] = np.array([case.load.loc[:,'q_mvar']])

Data_form_1 = np.hstack((P_Gen,Load))/S_base  

           
### Data_form_2: R X  

R = np.multiply(np.array([case.line.loc[:,'r_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()
X = np.multiply(np.array([case.line.loc[:,'x_ohm_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()
C = np.multiply(np.array([case.line.loc[:,'c_nf_per_km']]),np.array([case.line.loc[:,'length_km']])).transpose()*1e-9
C[C!=0] = 1/(2*np.pi*Freq*C[C!=0])
Xc = C

Data_form_2 = np.hstack((np.multiply(R,1/Z_base),\
                       np.multiply(X,1/Z_base),\
                        np.multiply(-Xc,1/Z_base)))



## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [4]:
number_of_data = 10000       # Including data for both Training and Testing

Scenario = np.zeros([number_of_data,Data_form_1.shape[0],Data_form_1.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form_1
    Scenario[i,np.unique(np.nonzero(Load)[0]),1:] = norm.ppf(np.random.rand(load_bus_number.shape[0],2),\
                                            Load[np.unique(np.nonzero(Load)[0]),:],\
                                            np.multiply(0.1,Load[np.unique(np.nonzero(Load)[0]),:]))/S_base

## Implement Power Flow to Get Labels

In [6]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],1:]*S_base
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]

## Exporting Data to CSV files

In [12]:
Power_Data = pd.DataFrame(Scenario.reshape(number_of_data,-1))
Power_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/Data_30bus/Dataset_30bus_LoadGeneration.csv", index = False)

Lines_Data = pd.DataFrame(Data_form_2)
Power_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/Data_30bus/Dataset_30bus_Lines.csv", index = False)

Final_Label = pd.DataFrame(Label.reshape(number_of_data,-1))
Final_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/Data_30bus/Labels_30bus.csv", index = False)



# Final_Test_Data   = pd.DataFrame(Test_Data.reshape(number_of_test_data,-1))
# Final_Test_Label  = pd.DataFrame(Test_Label.reshape(number_of_test_data,-1))
# Final_Test_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_data_30bus.csv",index = False)
# Final_Test_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_label_30bus.csv",index = False)